<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/flux/GUFlux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grand Unified Flux
With much love to @camenduru for the [flux_jupyter](https://github.com/camenduru/flux-jupyter) repository.

In [ ]:
#@title _Code be in this here box_
%cd /content
!git clone -b totoro4 https://github.com/camenduru/ComfyUI /content/TotoroUI
%cd /content/TotoroUI

!pip install -q torchsde einops diffusers accelerate xformers==0.0.28.post2 gradio==3.50.2 python-multipart==0.0.12
!apt -y install -qq aria2

# Install torchvision with CUDA support
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-dev-fp8-all-in-one.safetensors -d /content/TotoroUI/models/checkpoints -o flux1-dev-fp8-all-in-one.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux_realism_lora.safetensors -d /content/TotoroUI/models/loras -o flux_realism_lora.safetensors

import math
import random
import torch
import numpy as np
from PIL import Image
from PIL.PngImagePlugin import PngInfo
import nodes
from nodes import NODE_CLASS_MAPPINGS
from totoro_extras import nodes_custom_sampler
from totoro_extras import nodes_flux
from totoro import model_management
import gradio as gr
import os

#def add_ai_metadata(image_path,seed,prompt):
#  """
#  Adds AI generation metadata (seed and prompt) to an image.
#
#  Args:
#      image_path (str): Path to the image file.
#      seed (int): Random seed used for the image generation.
#      prompt (str): Text prompt used for the image generation.
#
#  Returns:
#      None
#  """
#  with Image.open(image_path) as img:
#    metadata = img.info
#    metadata["seed"] = str(seed)
#    metadata["prompt"] = prompt
#    img.save(image_path, **metadata)
#    print(f"AI generation metadata added to {image_path}")

def add_ai_metadata(image_path, metadata):
  """Adds metadata to a PNG image.

  Args:
    image_path: Path to the image file.
    metadata: A dictionary containing key-value pairs of metadata.
  """

  targetImage = Image.open("pathToImage.png")
  metadata = PngInfo()
  metadata.add_text("MyNewString", "A string")
  metadata.add_text("MyNewInt", str(1234))

  targetImage.save("NewPath.png", pnginfo=metadata)
  print(f"AI generation metadata added to {image_path}")

def optimal_dimensions(wh_ratio):
    """
    Calculates optimal dimensions for Stable Diffusion.

    Args:
      wh_ratio (str): Width-to-height ratio in the format "width:height".

    Returns:
      A tuple of (new_width, new_height) representing the optimal dimensions.
    """
    sw, sh = wh_ratio.split(':')
    w, h = int(sw), int(sh)
    c = math.sqrt(1024**2 / (w * h))
    new_width = int(((w * c) // 16) * 16)
    new_height = int(((h * c) // 16) * 16)
    #print(f"Optimal dimensions: {new_width}x{new_height}")
    return new_width, new_height

CheckpointLoaderSimple = NODE_CLASS_MAPPINGS["CheckpointLoaderSimple"]()
LoraLoader = NODE_CLASS_MAPPINGS["LoraLoader"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
RandomNoise = nodes_custom_sampler.NODE_CLASS_MAPPINGS["RandomNoise"]()
BasicGuider = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicGuider"]()
KSamplerSelect = nodes_custom_sampler.NODE_CLASS_MAPPINGS["KSamplerSelect"]()
BasicScheduler = nodes_custom_sampler.NODE_CLASS_MAPPINGS["BasicScheduler"]()
SamplerCustomAdvanced = nodes_custom_sampler.NODE_CLASS_MAPPINGS["SamplerCustomAdvanced"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet, clip, vae = CheckpointLoaderSimple.load_checkpoint("flux1-dev-fp8-all-in-one.safetensors")

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

@torch.inference_mode()
#def generate(positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
def generate(positive_prompt, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    global unet, clip
    if seed == 0:
        seed = random.randint(0, 18446744073709551615)
    print(f"\nSeed:  {seed}")
    #print(f"Prompt:  {positive_prompt}")
    width, height = optimal_dimensions(wh_ratio)
    if orientation == "portrait":
        width, height = height, width
    print(f"Dimentions:  {width}x{height} ({orientation})")
    #print(f"Width:  {width}\tType:  {type(width)}")
    #print(f"Height:  {height}\tType:  {type(height)}")
    #print(f"Steps:  {steps}\tType:  {type(steps)}
    unet_lora, clip_lora = LoraLoader.load_lora(unet, clip, "flux_realism_lora.safetensors", lora_strength_model, lora_strength_clip)
    cond, pooled = clip_lora.encode_from_tokens(clip_lora.tokenize(positive_prompt), return_pooled=True)
    cond = [[cond, {"pooled_output": pooled}]]
    cond = FluxGuidance.append(cond, guidance)[0]
    noise = RandomNoise.get_noise(seed)[0]
    guider = BasicGuider.get_guider(unet_lora, cond)[0]
    sampler = KSamplerSelect.get_sampler(sampler_name)[0]
    sigmas = BasicScheduler.get_sigmas(unet_lora, scheduler, steps, 1.0)[0]
    latent_image = EmptyLatentImage.generate(closestNumber(width, 16), closestNumber(height, 16))[0]
    sample, sample_denoised = SamplerCustomAdvanced.sample(noise, guider, sampler, sigmas, latent_image)
    decoded = VAEDecode.decode(vae, sample)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save("/content/flux.png")
    metadata = {
        "prompt": positive_prompt,
        "seed": seed,
        "sampler": sampler_name,
        "scheduler": scheduler,
        "guidance": guidance,
        "lora_strength_model": lora_strength_model,
        "lora_strength_clip": lora_strength_clip
    }
    #add_ai_metadata("/content/flux.png", metadata)
    return "/content/flux.png"

def generate_wrapper(positive_prompt, quantity, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip):
    for i in range(quantity):
        print(f"----\nGenerating image {i+1}/{quantity}...")
        generate(positive_prompt, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip)
        os.rename("/content/flux.png", f"/content/flux{i+1}.png")
    return f"/content/flux{i+1}.png"

with gr.Blocks(analytics_enabled=False) as demo:
    with gr.Row():
        with gr.Column():
            positive_prompt = gr.Textbox(lines=3, interactive=True, value="Anime drawing, full body portrait, attractive 19-year-old Caucasian woman, long straight blonde hair, red lipstick, white button-up blouse, black neck tie, black suspenders, tan suit jacket, tan shorts, tan pantyhose, tan flat shoes, smiling, sitting in a recliner, legs crossed", label="Prompt")
            #width = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="width")
            #height = gr.Slider(minimum=256, maximum=2048, value=1024, step=16, label="height")
            with gr.Row():
                #wh_ratio = gr.Textbox(lines=1, interactive=True, value="4:7", label="width:height ratio")
                wh_ratio = gr.Dropdown(["1:1","2:1","3:2","5:3","7:4","9:7","16:9","21:11","17:15"], value="7:4", label="width:height ratio")
                orientation = gr.Dropdown(["portrait", "landscape"], label="orientation", value="portrait")
                quantity = gr.Slider(minimum=1, maximum=10, value=1, step=1, label="quantity")
                #width, height = optimal_dimensions(wh_ratio.value)
            seed = gr.Slider(minimum=0, maximum=18446744073709551615, value=0, step=1, label="seed (0=random)")
            steps = gr.Slider(minimum=4, maximum=50, value=20, step=1, label="steps")
            guidance = gr.Slider(minimum=0, maximum=20, value=3.5, step=0.5, label="guidance")
            lora_strength_model = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_model")
            lora_strength_clip = gr.Slider(minimum=0, maximum=1, value=1.0, step=0.1, label="lora_strength_clip")
            sampler_name = gr.Dropdown(["euler", "heun", "heunpp2", "heunpp2", "dpm_2", "lms", "dpmpp_2m", "ipndm", "deis", "ddim", "uni_pc", "uni_pc_bh2"], label="sampler_name", value="euler")
            scheduler = gr.Dropdown(["normal", "sgm_uniform", "simple", "ddim_uniform"], label="scheduler", value="simple")
            generate_button = gr.Button("Generate")
        with gr.Column():
            output_image = gr.Image(label="Generated image", interactive=False)

    #generate_button.click(fn=generate, inputs=[positive_prompt, width, height, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)
    generate_button.click(fn=generate_wrapper, inputs=[positive_prompt, quantity, wh_ratio, orientation, seed, steps, sampler_name, scheduler, guidance, lora_strength_model, lora_strength_clip], outputs=output_image)

demo.queue().launch(inline=False, share=True, debug=True)